## Rò rỉ chỉ xảy ra nếu:

- Tập validation bị sử dụng trực tiếp trong quá trình tối ưu hóa mô hình.
- Hoặc bạn kết hợp train và validation mà không tách biệt đúng cách.

Để kiểm tra xem **tập validation có bị sử dụng trực tiếp trong quá trình tối ưu hóa mô hình không**, cần xem xét kỹ từng phần trong đoạn code:

---

### 1. **Trong `TrainerCustom.compute_loss`**
Phương thức `compute_loss` thực hiện các bước sau:
- Lấy dữ liệu đầu vào (`inputs`) và tách nhãn (`labels`) nếu có.
- Tính toán loss bằng cách so sánh dự đoán (`logits`) của mô hình với `labels` bằng `nn.CrossEntropyLoss`.

**Quan trọng**:
- `compute_loss` được gọi trong quá trình huấn luyện (`train`), và dữ liệu đầu vào cho nó được lấy từ `train_dataset`.
- **Không có bất kỳ đoạn nào sử dụng `test_dataset` hoặc tập validation trong `compute_loss`**.

---

### 2. **Trong `TrainerCustom.evaluate`**
Phương thức `evaluate` chỉ được gọi sau khi hoàn thành một epoch hoặc tại các điểm đánh giá theo chiến lược (`eval_strategy="epoch"`). Nó:
- Sử dụng `test_dataset` để tính `eval_loss`.
- **Không thực hiện cập nhật trọng số mô hình** khi tính `eval_loss`.

Do đó, không có rò rỉ nào xảy ra ở đây.

---

### 3. **Trong `TrainerCustom.train`**
Phương thức `train` gọi `compute_loss` để tính toán loss cho từng batch và cập nhật trọng số mô hình. Dữ liệu đầu vào cho `compute_loss` đến từ `train_dataset`. 

- `train_dataset` được sử dụng để tối ưu hóa mô hình.
- **Không có chỗ nào sử dụng `test_dataset` trong phương thức `train` hoặc quá trình tối ưu hóa**.

---

### 4. **Sử dụng tập validation trong cập nhật trọng số**
Để xác nhận rõ hơn, bạn có thể thêm các đoạn log kiểm tra trong `compute_loss` và `evaluate` để đảm bảo rằng dữ liệu từ `test_dataset` không bị sử dụng trong tối ưu hóa:

#### Kiểm tra trong `compute_loss`:
```python
def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
    if inputs.get("dataset") == "test":
        print("WARNING: Validation data used in compute_loss!")
    ...
```

#### Kiểm tra trong `evaluate`:
```python
def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
    print(f"Evaluating with {metric_key_prefix}, dataset: {eval_dataset}")
    return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
```

---

### 5. **Cách xác định vấn đề rò rỉ:**
Rò rỉ xảy ra nếu:
1. **`test_dataset` bị sử dụng trong quá trình huấn luyện.** Điều này có thể xảy ra nếu `test_dataset` vô tình được đưa vào trong `train()` hoặc `compute_loss`.
2. **`train_dataset` và `test_dataset` không được tách biệt đúng cách**. Ví dụ, nếu `train_dataset` và `test_dataset` có mẫu trùng nhau, điều đó dẫn đến rò rỉ.

Bạn có thể thêm đoạn kiểm tra:
```python
# Kiểm tra xem train_dataset và test_dataset có trùng mẫu không
train_ids = set(train_dataset["id"])  # Nếu dataset có trường "id"
test_ids = set(test_dataset["id"])

overlap = train_ids.intersection(test_ids)
if len(overlap) > 0:
    print(f"WARNING: {len(overlap)} samples are shared between train and test datasets!")
```

---

### 6. **Kết luận:**
Dựa trên code bạn cung cấp:
- **Tập validation không bị sử dụng trực tiếp trong tối ưu hóa mô hình**.
- Đảm bảo rằng `train_dataset` và `test_dataset` được tách biệt đúng cách.

Nếu bạn thấy tập validation bị sử dụng trong tối ưu hóa, hãy kiểm tra kỹ log hoặc quy trình xử lý dữ liệu ban đầu (`train_test_split` hoặc pipeline dữ liệu).

```python
from concurrent.futures import ThreadPoolExecutor
import wandb
import os
import shutil
import time

class TrainerCustom(Trainer):
    def __init__(self, *args, save_every_n_epochs=10, **kwargs):
        super().__init__(*args, **kwargs)
        if torch.cuda.is_available():
            print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
        else:
            print("Trainer is running on CPU.")

        self.best_eval_loss = float("inf")  # Giá trị loss tốt nhất ban đầu
        self.save_every_n_epochs = save_every_n_epochs  # Tần suất lưu lên WandB
        self.best_model_info = {"epoch": None, "loss": None}
        self.last_saved_epoch = 0  # Epoch cuối cùng đã lưu Best Model và Last Model
        self.executor = ThreadPoolExecutor(max_workers=3)  # Cho phép tối đa 2 luồng song song

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """

        # # Kiểm tra thiết bị của mô hình và dữ liệu
        # print("Model device:", next(model.parameters()).device)
        # print("Input device:", inputs["input_ids"].device)
        if "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        # Sử dụng nn.CrossEntropyLoss() thay vì nn.CrossEntropy
        cross_entropy_loss = nn.CrossEntropyLoss()

        # Chạy mô hình và nhận đầu ra (logits)
        outputs = model(**inputs)

        # Đảm bảo lấy logits từ outputs (mô hình trả về tuple, lấy phần tử đầu tiên là logits)
        logits = outputs

        if labels is None:
            print("Labels are None during compute_loss.")
        if logits is None:
            print("Logits are None during compute_loss.")

        # Tính toán loss
        loss = cross_entropy_loss(logits, labels)

        # Trả về loss và outputs nếu cần
        return (loss, outputs) if return_outputs else loss

    def async_save_model(self, model_dir, artifact_name, metadata=None):
        """
        Lưu mô hình vào local và đồng bộ lên WandB trong luồng song song.
        """
        def save():
            start_time = time.time()
            try:
                # Xóa tất cả các thư mục tmp_best_model_ trước đó
                for folder in os.listdir("."):
                    if folder.startswith("tmp_best_model_epoch_") and folder != model_dir:
                        shutil.rmtree(folder, ignore_errors=True)
                        print(f"Removed old temporary directory: {folder}")

                # Lưu mô hình vào thư mục tạm
                self.save_model(model_dir)

                # Đồng bộ lên WandB
                artifact = wandb.Artifact(artifact_name, type="model")
                artifact.add_dir(model_dir)
                if metadata:
                    artifact.metadata = metadata
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"Error during saving or syncing model {artifact_name}: {e}")
            finally:
                # Xóa thư mục tạm hiện tại sau khi đồng bộ
                try:
                    shutil.rmtree(model_dir, ignore_errors=True)
                    print(f"Successfully removed temporary directory: {model_dir}")
                except Exception as e:
                    print(f"Error removing temporary directory {model_dir}: {e}")

            elapsed_time = time.time() - start_time
            print(f"Model saved and uploaded to WandB: {artifact_name} in {elapsed_time:.2f} seconds")

        self.executor.submit(save)




    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix: str = "eval"):
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
        eval_loss = metrics.get("eval_loss")

        # Cập nhật Best Model nếu eval_loss giảm
        # Lưu Best Model ngay khi eval_loss giảm (local).
        # Đồng bộ wandb ngay

        if eval_loss is not None and eval_loss < self.best_eval_loss:
            print(f"New best eval_loss: {eval_loss}")
            self.best_eval_loss = eval_loss
            self.best_model_info = {"epoch": self.state.epoch, "loss": eval_loss}

            # Log thông tin Best Model lên WandB
            wandb.log({
                "best_eval_loss": self.best_eval_loss,
                "best_model_epoch": self.best_model_info.get("epoch", -1)
            })

            # Lưu Best Model vào thư mục tạm (local)
            best_model_dir = f"./tmp_best_model_epoch_{int(self.state.epoch)}"
            self.save_model(best_model_dir)

            # Đồng bộ lên WandB ngay
            artifact_name = f"best_model_epoch_{int(self.state.epoch)}"
            self.async_save_model(best_model_dir, artifact_name, self.best_model_info)

        return metrics

    # def save_last_model(self):
    #     """
    #     Lưu Last Model lên WandB sau mỗi N epochs.
    #     """
    #     if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
    #         print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")
    #         last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
    #         artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
    #         self.async_save_model(last_model_dir, artifact_name)

    #         # Log thông tin Last Model lên WandB
    #         wandb.log({
    #             "last_model_epoch": self.state.epoch
    #         })

    #         # Cập nhật epoch cuối cùng đã lưu
    #         self.last_saved_epoch = int(self.state.epoch)

    def save_last_model(self):
        """
        Lưu Last Model (bao gồm trạng thái optimizer, scheduler) lên WandB sau mỗi N epochs.
        """
        if int(self.state.epoch) % self.save_every_n_epochs == 0 and int(self.state.epoch) != self.last_saved_epoch:
            print(f"Saving Last Model at epoch {self.state.epoch} to WandB...")

            # Thư mục tạm lưu checkpoint
            last_model_dir = f"./tmp_last_model_epoch_{int(self.state.epoch)}"
            os.makedirs(last_model_dir, exist_ok=True)

            # Lưu đầy đủ trạng thái mô hình (checkpoint)
            self.save_model(last_model_dir)

            # Đường dẫn tệp trainer_state.json
            trainer_state_path = os.path.join(last_model_dir, "trainer_state.json")
            self.state.save_to_json(trainer_state_path)  # Lưu trạng thái trainer

            # Đồng bộ checkpoint lên WandB
            artifact_name = f"last_model_epoch_{int(self.state.epoch)}"
            metadata = {
                "epoch": int(self.state.epoch),
                "last_eval_loss": self.state.best_metric if hasattr(self.state, "best_metric") else "N/A",
            }
            self.async_save_model(last_model_dir, artifact_name, metadata)

            # Cập nhật epoch cuối cùng đã lưu
            self.last_saved_epoch = int(self.state.epoch)


    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)

        # Sau mỗi epoch, lưu Last Model lên WandB
        self.save_last_model()
        # Chờ tất cả các luồng lưu hoàn thành trước khi kết thúc
        self.executor.shutdown(wait=True)

        return result


# Bước 6: Cài đặt tham số huấn luyện
training_args = TrainingArguments(
    output_dir="./result__s",          # Thư mục lưu kết quả
    eval_strategy="epoch",    # Đánh giá sau mỗi epoch
    learning_rate=2e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=1,  # Ghi logs mỗi 500 bước huấn luyện
    save_strategy="no",          # Lưu trọng số sau mỗi epoch
    save_total_limit=3,
    label_names = ["labels"],
    report_to="wandb",
    run_name="bert_run_3"
)


import wandb

# Khởi tạo wandb
wandb.init(
    project="bertIntentClassification",  # Tên dự án
    name="bert_10000Data_1epoch",                     # Tên phiên chạy
    config={"gpu": torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else "CPU"}
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if torch.cuda.is_available():
    print(f"Trainer is running on GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("Trainer is running on CPU.")

trainer = TrainerCustom(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=collate_fn,
    save_every_n_epochs=3  # Lưu Best Model và Last Model mỗi 10 epochs
)

trainer.train()


wandb.finish()
```